# In this notebook we will run through...
How Lasso writes out EMME network
1. Read Network Wrangler Standard Network
2. Create model network from standard network
3. Write out EMME network from model network

In [15]:
import os

import pytest
import pickle
import copy

from lasso import Parameters, transit
#from lasso import emme
from network_wrangler import TransitNetwork, RoadwayNetwork
from lasso import StandardTransit
from lasso import ModelRoadwayNetwork
from lasso import mtc

In [3]:
%load_ext autoreload
%autoreload 2

# Remote I/O

In [4]:
lasso_dir = os.path.join('Z:/Data/Users/Sijia/Met_Council/github/client_met_council_wrangler_utilities')

input_dir = os.path.join(lasso_dir, 'examples', 'mtc')

In [5]:
output_dir = os.path.join(lasso_dir, 'tests', 'scratch')

In [6]:
parameters = Parameters(lasso_base_dir = lasso_dir)

2022-05-23 10:47:22, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/Met_Council/github/client_met_council_wrangler_utilities
2022-05-23 10:47:22, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/Met_Council/github/client_met_council_wrangler_utilities


## Union city network files

In [10]:
link_file = os.path.join(input_dir, 'link.json')
node_file = os.path.join(input_dir, 'node.geojson')
shape_file = os.path.join(input_dir, 'shape.geojson')

road_net = RoadwayNetwork.read(
    link_filename = link_file, 
    node_filename = node_file, 
    shape_filename = shape_file
)

2022-05-23 11:40:06, INFO: Reading RoadwayNetwork
2022-05-23 11:40:06, DEBUG: Reading RoadwayNetwork from following files:
   -Z:/Data/Users/Sijia/Met_Council/github/client_met_council_wrangler_utilities\examples\mtc\link.json
   -Z:/Data/Users/Sijia/Met_Council/github/client_met_council_wrangler_utilities\examples\mtc\node.geojson
   -Z:/Data/Users/Sijia/Met_Council/github/client_met_council_wrangler_utilities\examples\mtc\shape.geojson.
2022-05-23 11:40:10, INFO: Read 16411 links from Z:/Data/Users/Sijia/Met_Council/github/client_met_council_wrangler_utilities\examples\mtc\link.json
2022-05-23 11:40:10, INFO: Read 5233 nodes from Z:/Data/Users/Sijia/Met_Council/github/client_met_council_wrangler_utilities\examples\mtc\node.geojson
2022-05-23 11:40:10, INFO: Read 8514 shapes from Z:/Data/Users/Sijia/Met_Council/github/client_met_council_wrangler_utilities\examples\mtc\shape.geojson


In [13]:
model_net = ModelRoadwayNetwork.from_RoadwayNetwork(
    roadway_network_object = road_net, 
    parameters = parameters)

2022-05-23 11:42:06, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/Met_Council/github/client_met_council_wrangler_utilities
2022-05-23 11:42:06, INFO: Lasso base directory set as: Z:/Data/Users/Sijia/Met_Council/github/client_met_council_wrangler_utilities
2022-05-23 11:42:06, INFO: Filling nan for network from network wrangler
2022-05-23 11:42:06, INFO: Converting variable type to mtc standard


In [17]:
# there are a few steps needed to create travel model network from standard network
# we will not run here since these steps were ran at the full network creation
# e.g. add centroid connectors, add mpo specific attributes, convert to mpo crs...

# model_net = mtc.add_centroid_and_centroid_connector(
#    roadway_network = model_net,
#    parameters = parameters
# )
# model_net = mtc.add_tap_and_tap_connector(
#    roadway_network = model_net,
#    transit_network = union_city_transit_net, 
#    parameters = parameters
# )
# model_net = mtc.roadway_standard_to_mtc_network(model_net, parameters)

# write model_net.links_mtc_df to pickle file union_city_links.pickle
# write model_net.nodes_mtc_df to pickle file union_city_nodes.pickle

In [18]:
# union city model network 

# roadway
union_city_links_df = pickle.load(open(os.path.join(input_dir, "union_city_links.pickle"), 'rb'))
union_city_nodes_df = pickle.load(open(os.path.join(input_dir, "union_city_nodes.pickle"), 'rb'))

In [19]:
# transit
transit_net = TransitNetwork.read(feed_path = input_dir)

# transit netowrk for union city
agency_df = transit_net.feed.agency.loc[transit_net.feed.agency['agency_id'] == 'UCT']
routes_df = transit_net.feed.routes.loc[transit_net.feed.routes['agency_id'] == 'UCT']
trips_df = transit_net.feed.trips[transit_net.feed.trips['route_id'].isin(routes_df['route_id'])]
freqs_df = transit_net.feed.frequencies[transit_net.feed.frequencies['trip_id'].isin(trips_df['trip_id'])]
shapes_df = transit_net.feed.shapes[transit_net.feed.shapes['shape_id'].isin(trips_df['shape_id'])]
stop_times_df = transit_net.feed.stop_times[transit_net.feed.stop_times['trip_id'].isin(trips_df['trip_id'])]
stops_df = transit_net.feed.stops[transit_net.feed.stops['stop_id'].isin(stop_times_df['stop_id'])]

union_city_transit_net =  copy.deepcopy(transit_net)
union_city_transit_net.feed.agency = agency_df
union_city_transit_net.feed.routes = routes_df
union_city_transit_net.feed.trips  = trips_df
union_city_transit_net.feed.frequencies = freqs_df
union_city_transit_net.feed.shapes = shapes_df
union_city_transit_net.feed.stop_times = stop_times_df
union_city_transit_net.feed.stops = stops_df

model_transit_net = StandardTransit.fromTransitNetwork(union_city_transit_net, parameters = parameters)

2022-05-23 11:50:02, INFO: Read in transit feed from: Z:/Data/Users/Sijia/Met_Council/github/client_met_council_wrangler_utilities\examples\mtc
2022-05-23 11:50:03, DEBUG: ...agency.txt:
        agency_id                    agency_name  \
0            BART         Bay Area Rapid Transit   
1              CE  ACE Altamont Corridor Express   
2             NaN              County Connection   
3  caltrain-ca-us                       Caltrain   
4             573                 Emery Go-Round   
5              36   Fairfield and Suisun Transit   
6             NaN            Golden Gate Transit   
7             533             MVgo Mountain View   
8             NaN    Stanford Marguerite Shuttle   
9            MCTD                  Marin Transit   

                         agency_url      agency_timezone agency_lang  \
0               http://www.bart.gov  America/Los_Angeles          en   
1           http://www.acerail.com/  America/Los_Angeles          en   
2  http://www.countyconn

2022-05-23 11:50:03, DEBUG: ...calendar_dates.txt:
Empty DataFrame
Columns: [service_id, date, exception_type]
Index: []
2022-05-23 11:50:03, INFO: Removing calendar_dates.txt from transit network config because file not found
2022-05-23 11:50:05, DEBUG: ...fare_attributes.txt:
  fare_id  price currency_type  payment_method transfers  transfer_duration  \
0       1   2.00           USD             0.0       NaN             7200.0   
1       3   0.00           USD             0.0       NaN             7200.0   
2       2   2.25           USD             0.0       NaN             7200.0   
3       1   2.25           USD             0.0       NaN             5400.0   
4       2   6.00           USD             0.0       0.0                NaN   
5       1   4.75           NaN             NaN       NaN                NaN   
6       2   5.25           NaN             NaN       NaN                NaN   
7       3   6.50           NaN             NaN       NaN                NaN   
8       1 

2022-05-23 11:50:06, DEBUG: ...transfers.txt:
Empty DataFrame
Columns: [from_stop_id, to_stop_id, transfer_type]
Index: []
2022-05-23 11:50:06, INFO: Removing transfers.txt from transit network config because file not found
2022-05-23 11:50:06, DEBUG: ...feed_info.txt:
Empty DataFrame
Columns: [feed_publisher_name, feed_publisher_url, feed_lang]
Index: []
2022-05-23 11:50:06, INFO: Removing feed_info.txt from transit network config because file not found
2022-05-23 11:50:06, WARNING: The following stop_ids from are referenced but missing from stops.txt: ['place_12BD', 'place_BDBL', 'ctgi', 'cthi', 'ctca', 'place_FBBA', 'place_90MA', 'place_CCCO', 'place_MBLS', 'ctmh', 'place_4SSP', 'ctco', 'ctsmat', 'place_14BD', 'ctsj', 'place_11JE', 'ctpa', 'ct22', 'place_SHBA', 'ctbu', 'place_HPNR', 'place_CSSH', 'place_SHDF', 'ctsf', 'ctbl', 'place_CVBA', 'place_HRSH', 'ctbe', 'ctrwc', 'place_WOBA', 'ctsc', 'ctmi', 'place_HIWY', 'ctsmar', 'place_SMSE', 'ctsa', 'ctmp', 'place_HBBA', 'place_ROBA', 'c

# Write Emme Networks

In [12]:
emme.create_emme_network(
    links_df=union_city_links_df,
    nodes_df=union_city_nodes_df,
    transit_network=model_transit_net,
    name="test lasso",
    path=output_dir,
    write_taz_drive_network = False,
    write_maz_drive_network = False,
    write_maz_active_modes_network = False,
    write_tap_transit_network = True,
    parameters=parameters
)

2021-09-29 12:11:09, INFO: Converting GTFS Standard Properties to MTC's Emme Standard
2021-09-29 12:11:09, INFO: Reading faresystem from Z:/Data/Users/Sijia/MTC/github/Lasso/mtc_data\lookups\faresystem_crosswalk.txt
z:\data\users\sijia\mtc\github\lasso\tests\scratch\tap_transit_network\database\emmebank
